In [15]:
from math import floor
epsilon = 0.01
sub_epsilon = 50
hash_size = 64

utilization = [0] * hash_size
stored = 0
new_stored = 0
found = 0
hash_table = [(0,0,0,0,0,0)] * hash_size

i = 0

iterations = 2888

max_acc_x_ = 0
max_sub_epsilon = 0

iteration = 0

def theSame(f1, f2):
    return abs(f1 - f2) < (f1*sub_epsilon)
    
def theSameTuple(f1, f2):
    return theSame(f1[0], f2[0]) and theSame(f1[1], f2[1]) and theSame(f1[2], f2[2]) \
        and theSame(f1[3], f2[3]) and theSame(f1[4], f2[4]) and theSame(f1[5], f2[5])

with open("input/input_acc.csv") as f1:
    with open("input/input_gyro.csv") as f2:
        for line1, line2 in zip(f1, f2):
            iteration += 1
            acc_x = line1.split(",")[1].strip()
            acc_y = line1.split(",")[2].strip()
            acc_z = line1.split(",")[3].strip()
            
            gyro_x = line2.split(",")[1].strip()
            gyro_y = line2.split(",")[2].strip()
            gyro_z = line2.split(",")[3].strip()

            if i > 0:
                acc_x_ = float(acc_x)
                acc_y_ = float(acc_y)
                acc_z_ = float(acc_z)
                gyro_x_ = float(gyro_x)
                gyro_y_ = float(gyro_y)
                gyro_z_ = float(gyro_z)
                
                acc_x_hash = int(acc_x_ / epsilon)
                acc_y_hash = int(acc_y_ / epsilon)
                acc_z_hash = int(acc_z_ / epsilon)
                gyro_x_hash = int(gyro_x_ / epsilon)
                gyro_y_hash = int(gyro_y_ / epsilon)
                gyro_z_hash = int(gyro_z_ / epsilon)

                hash_ = (acc_x_hash + acc_y_hash + acc_z_hash + gyro_x_hash + gyro_y_hash + gyro_z_hash) % hash_size
                
                utilization[hash_] += 1
                
                #print("Comparison in-between: " + str((acc_x_,acc_y_,acc_z_,gyro_x_,gyro_y_,gyro_z_)) + ", " + str(hash_table[hash_]))
                
                if hash_table[hash_] == (0,0,0,0,0,0):
                    hash_table[hash_] = (acc_x_, acc_y_, acc_z_, gyro_x_, gyro_y_, gyro_z_)
                    new_stored += 1
                elif theSameTuple((acc_x_,acc_y_,acc_z_,gyro_x_,gyro_y_,gyro_z_), hash_table[hash_]):
                    found += 1
                else:
                    hash_table[hash_] = (acc_x_, acc_y_, acc_z_, gyro_x_, gyro_y_, gyro_z_)
                    stored += 1

                if i < 6: print("{0} -> {1} {2}".format(hash_, acc_x, acc_x_*sub_epsilon))
                max_acc_x_ = max(max_acc_x_, acc_x_)
                max_sub_epsilon = max(max_sub_epsilon, abs(acc_x_ * sub_epsilon))
            
            i += 1
            if (i > iterations): break

26 -> 0.19296300000000002 9.648150000000001
2 -> 0.326981 16.349050000000002
0 -> 0.321121 16.05605
11 -> 0.400803 20.04015
2 -> 0.419296 20.9648


In [16]:
print("The maximum accX found: {0} (times sub_epsilon {1} is {2})".format(max_acc_x_, sub_epsilon, max_sub_epsilon))
print("")
print("HashTable population: " + str(utilization))
print("No. of times found: " + str(found))
print("No. of times stored: " + str(new_stored + stored) + " (New: {0}, Overwritten: {1})".format(new_stored, stored))

The maximum accX found: 1.6690060000000002 (times sub_epsilon 50 is 83.45030000000001)

HashTable population: [43, 51, 49, 48, 41, 38, 52, 49, 52, 48, 39, 46, 40, 41, 50, 38, 45, 57, 41, 36, 48, 54, 40, 50, 40, 49, 39, 51, 41, 49, 57, 40, 34, 50, 39, 42, 50, 38, 46, 37, 46, 44, 50, 49, 44, 39, 42, 41, 40, 44, 44, 46, 34, 43, 46, 54, 36, 54, 50, 43, 54, 54, 44, 49]
No. of times found: 131
No. of times stored: 2757 (New: 64, Overwritten: 2693)


In [17]:
print("Balance factor (index most used minus index least used, should be close to 0): " + str(max(utilization) - min(utilization)))
print("Most written in an index (means most overwrites for a single index, should be small): " + str(max(utilization)))

Balance factor (index most used minus index least used, should be close to 0): 23
Most written in an index (means most overwrites for a single index, should be small): 57


In [18]:
sum(utilization)

2888

In [19]:
print(iteration)

2889


In [20]:
import os
from experiment.helpers import *

verbose=2

props_and_values = {
    "HASHSIZE": hash_size,
    "ITERATIONS": iterations,
    "EPSILON" : '{0:f}'.format(epsilon),
    "SUB_EPSILON" : '{0:f}'.format(sub_epsilon)
}

x86_kalman_dir = os.path.join(os.path.dirname(os.getcwd()),"KalmanC++")
x86_kalman_hash_header = os.path.join(x86_kalman_dir, "HashTable", "hashtable.h")

hashtable_header_file_path = x86_kalman_hash_header
hashtable_header_file_clone_path = os.path.join(x86_kalman_dir, "Originals", "hashtable.h")
replace_define_value_in_file(hashtable_header_file_clone_path, hashtable_header_file_path, props_and_values)
#replace_define_value_in_file(hashtable_header_file_clone_path, hashtable_header_file_path, "EPSILON", epsilon)
#replace_define_value_in_file(hashtable_header_file_clone_path, hashtable_header_file_path, "EPSILON2", epsilon2)

if verbose >= 2: print("File '" + hashtable_header_file_path + "' now contains:")
if verbose >= 2: print_file(hashtable_header_file_path, "#define HASHSIZE")
if verbose >= 2: print_file(hashtable_header_file_path, "#define EPSILON")
if verbose >= 2: print_file(hashtable_header_file_path, "#define SUB_EPSILON")
if verbose >= 2: print_file(hashtable_header_file_path, "#define ITERATIONS")

File 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanC++\HashTable\hashtable.h' now contains:
#define HASHSIZE 64
#define EPSILON 0.010000
#define SUB_EPSILON 50.000000
#define ITERATIONS 2888


In [21]:
compile_hash()
run_hash(verbose=verbose)

Epsilon: 0.010000
Sub epsilon: 50.000000
Hash size: 64

HashTable population: 64 / 64
No. of times found: 137
No. of times stored: 2751
